Since the notebook server was updated, the following commands need to be run in a terminal before proceeding:
```
> conda install -c conda-forge scikit-sparse && \
TEMPO2_PREFIX='/opt/nanograv' pip install libstempo[astropy] --upgrade
> pip install git+https://github.com/nanograv/enterprise.git --upgrade --user
> pip install git+https://github.com/nanograv/enterprise_extensions.git --upgrade --user
> pip install git+https://github.com/mtlam/PyPulse.git --upgrade --user
> pip install numdifftools
```
Also note: if you have existing pickled tim files in your working directory, remove them before running the notebook, otherwise you're likely to run into an issue with `alias_translation`. This may get simpler with more version tagging and when the docker container is rebuilt, but here we are. Don't forget to restart the kernel before proceeding.

In [ ]:
from timing_analysis.lite_utils import *
from timing_analysis.par_checker import *
from timing_analysis.utils import *
from timing_analysis.dmx_utils import *
from timing_analysis.timingconfiguration import TimingConfiguration
from timing_analysis.outlier_utils import *
from timing_analysis.utils import apply_cut_flag, apply_cut_select
import yaml
from astropy import log
import pint.fitter
from pint.utils import dmxparse
import os

# For manual inspection:
from ipywidgets import *
from IPython.display import Image, display

In [ ]:
config = "configs/[psr_name].[nb or wb].yaml"  # fill in actual path
par_directory = None   # default location
tim_directory = None   # default location
tc = TimingConfiguration(config, par_directory=par_directory, tim_directory=tim_directory)
using_wideband = tc.get_toa_type() == 'WB'

# For manual inspection:
epoch_matches, toa_matches = tc.get_investigation_files()

In [ ]:
run_outlier_analysis = False
run_epochalyptica = False
use_outlier_results = True

if run_outlier_analysis:
    mo,to = tc.get_model_and_toas(apply_initial_cuts=True)
    to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())
    tc.check_outlier()

    calculate_pout(mo, to, tc)
    make_pout_cuts(mo, to, tc, outpct_threshold=8.0)
    if run_epochalyptica: epochalyptica(mo,to,tc)

if use_outlier_results:
    tc.config['toas'] = find_excise_file(tc.get_outfile_basename())
    if tc.config['toas']:
        tc.tim_directory = ''
        mo,to = tc.get_model_and_toas(excised=True)
        tc.check_file_outliers(to, outpct_threshold=8.0)
        to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

In [ ]:
# Quick breakdown of existing cut flags (automated excision)
cuts_dict = cut_summary(to)

In [ ]:
# Apply manual cuts and check for redundancies
tc.manual_cuts(to)
to = setup_dmx(mo,to,frequency_ratio=tc.get_fratio(),max_delta_t=tc.get_sw_delay())

In [ ]:
# Plot automated cuts for inspection
from timing_analysis.plot_utils import *
fo = tc.construct_fitter(to,mo)
plot_residuals_time(fo, restype='prefit')

Running display_excise_dropdowns (as below) will cause dropdowns for each bad-epoch and bad-toa of interest to be displayed. You should choose whatever combinations of file and plot type you'd like to make, then run the cell after this one (containing the read_excise_dropdowns functions) to read in your choices. The optional arguments all_YFp, all_GTpd, and all_profile will default the checkboxes to those plot types. If not specified, the default will be 'None' (i.e. do not plot). Finally, the make_detective_plots function (next cell) will display your plots.

In [ ]:
# Display checkboxes
epoch_select_list, epoch_pav_list, toa_select_list, toa_pav_list = \
    display_excise_dropdowns(epoch_matches, toa_matches, all_YFp=False, all_GTpd=False, all_profile=False)

As described above, this block will read your choices from the above dropdowns and make plots. Bad epochs and bad toas are dealt with separately to allow for future use of the bad-toa subint/subband info.

In [ ]:
# Make plots (indicated by dropdown choices)
epoch_plot_list = read_excise_dropdowns(epoch_select_list, epoch_pav_list, epoch_matches)
toa_plot_list = read_excise_dropdowns(toa_select_list, toa_pav_list, toa_matches)
make_detective_plots(epoch_plot_list, epoch_matches)
make_detective_plots(toa_plot_list, toa_matches)